In [1]:
import numpy as np
import cv2
import cv2 as cv
from synthetic_dataset import *
from spherical_module import *

In [482]:
def draw_interest_points(img, points):
    """ Convert img in RGB and draw in green the interest points """
    img_rgb = np.stack([img, img, img], axis=2)
    for i in range(points.shape[0]):
        cv.circle(img_rgb, (points[i][0], points[i][1]), 5, (0, 255, 0), -1)
    return img_rgb

In [483]:
res_std = 256
#b_img = generate_background(size=(1024, 2048)) /255
b_img = generate_background(size=(res_std*2, res_std*4))
#cv.imshow('Random Lines', cv.resize(b_img, (1024, 512)))
#cv2.imwrite('b_img.png', b_img*255)
#cv.waitKey(0)
#cv.destroyAllWindows()

In [613]:
b_images, output_sqr = create_cube_imgs(b_img)
images = []
all_points_list = []
for idx, img in enumerate(b_images):
    if len(img.shape) == 3:
        img = img[:, :, 0]/255
    if img.shape != (res_std, res_std):
        img = cv.resize(img, (res_std, res_std))
    img *= 255
    #points = draw_polygon(img)
    #points = draw_lines(img)
    #points = draw_multiple_polygons(img)
    #points = draw_ellipses(img)
    #points = draw_star(img)
    #points = draw_checkerboard(img)
    #points = draw_stripes(img)
    #points = draw_cube(img)

    #points = draw_lines(img)
    #all_points_list.append(points)

    if idx == 1:
        points = draw_lines(img)
        all_points_list.append(points)
        cv.imshow('Random Lines', img)
        cv2.imwrite('aaa_img.png', img*255)
        cv.waitKey(0)
        cv.destroyAllWindows()

    #points = gaussian_noise(img)
    #print(points)
    if False:
        img = draw_interest_points(img, points)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    images.append(img)

In [614]:
# 6枚の画像を1枚のキューブマップ画像に結合する
cube_map_size = (res_std * 3, res_std * 4)  # キューブマップのサイズ
cube_map = np.zeros(cube_map_size, dtype=np.uint8)

positions = [(res_std, res_std), (res_std, 0), (res_std, res_std*3), 
             (res_std, res_std*2), (0, res_std), (res_std*2, res_std)]
for idx, (pos, img) in enumerate(zip(positions, images)):
    cube_map[pos[0]:pos[0]+res_std, pos[1]:pos[1]+res_std] = img

flag_cube_points_vis = False
if flag_cube_points_vis:
    for idx, (pos, img) in enumerate(zip(positions, images)):
        all_points_list[idx][:, 0] += pos[1]
        all_points_list[idx][:, 1] += pos[0]
    points = np.concatenate(all_points_list, axis=0)
    cube_map = draw_interest_points(cube_map, points)
    #cube_map = cv2.cvtColor(cube_map, cv2.COLOR_BGR2GRAY)
    cv.imshow('Random Lines', cv.resize(cube_map, (600, 400)))
    cv2.imwrite('cube_img.png', cube_map)
    cv.waitKey(0)
    cv.destroyAllWindows()
cv.imshow('Random Lines', cv.resize(cube_map, (600, 400)))
cv2.imwrite('cube_img.png', cube_map)
cv.waitKey(0)
cv.destroyAllWindows()

In [615]:
import numpy as np

def cube_to_equirectangular(x_cube, y_cube, idx, cube_size=256, equi_hw=(512, 1024)):
    face_idx = ["front", "left", "back", "right", "top", "bottom"]
    face = face_idx[idx]

    # キューブマップの中心からの相対座標
    x_rel = (x_cube - cube_size / 2) / (cube_size /2)
    y_rel = (y_cube - cube_size / 2) / (cube_size /2)
    
    # 各面に応じた変換
    if face == "front":
        theta = np.arctan2(x_rel, 1)  # 正面を向いているときのθ
        phi = np.arcsin(y_rel / np.sqrt(1+y_rel**2))
        print(phi)
    elif face == "top":
        theta = np.arctan2(y_rel, x_rel)
        phi = np.pi / 2
    elif face == "bottom":
        theta = np.arctan2(y_rel, x_rel)
        phi = -np.pi / 2
    elif face == "left":
        theta = np.arctan2(-x_rel, -1) - np.pi  # xとzの関係を用いてθを修正
        phi = np.arcsin(y_rel)
    elif face == "right":
        theta = np.arctan2(x_rel, 1)  # xとzの関係を用いてθを修正
        phi = np.arcsin(y_rel)
    elif face == "back":
        theta = np.arctan2(-x_rel, -1)  # 背面を向いているときのθ
        phi = np.arcsin(y_rel)
    else:
        return None
    
    # 球面座標から等角投影画像へのマッピング
    equi_height, equi_width = equi_hw
    x_equi = (theta + np.pi) / (2 * np.pi) * equi_width
    y_equi = (phi + np.pi/2) / np.pi * equi_height
    
    return x_equi, y_equi

In [616]:
cube_to_equirectangular(0, 0, 0)
cube_to_equirectangular(0, 256, 0)

-0.7853981633974482
0.7853981633974482


(384.0, 384.0)

In [617]:
points_sph = []
for idx in range(len(all_points_list)):
    points_tgt = all_points_list[idx]
    for (x_cube, y_cube) in points_tgt:
        x_sph, y_sph = cube_to_equirectangular(x_cube, y_cube, idx)
        #print(x_cube, y_cube)
        #print(x_sph, y_sph)
        x_sph = int(x_sph)
        y_sph = int(y_sph)
        points_sph.append([x_sph, y_sph])
points_sph = np.array(points_sph)

-0.18534794999569476
-0.5415676053918449


In [618]:
print(all_points_list)

[array([[ 69, 104],
       [140,  51]])]


In [619]:
eq_img = cube_to_equirectangular_np(cube_map, res_std*4)
print(eq_img.shape)
if True:
    eq_img = draw_interest_points(eq_img, points_sph)
    #eq_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv.imshow('Random Lines', cv.resize(eq_img, (res_std*4, res_std*2)))
    cv2.imwrite('eq_img.png', eq_img*255)
    cv.waitKey(0)
    cv.destroyAllWindows()

(512, 1024)


In [611]:
points

array([[ 33, 255],
       [242, 253]])

In [612]:
#b_img = generate_background(size=(1024, 2048)) /255
##cv.imshow('Random Lines', b_img)
#cv.waitKey(0)
#cv.destroyAllWindows()

In [492]:
#result_image_ndarray = np.where(eq_img*255 == 254, b_img, eq_img)
#result = cv2.resize(result_image_ndarray, (1024, 512))
#cv.imshow('Random Lines', result)
#cv2.imwrite('result.png', result*255)
#cv.waitKey(0)
#cv.destroyAllWindows()